In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map

NameError: name 'torch' is not defined

In [ ]:
if torch.cuda.is_available():  # Fallback to CUDA if available
    device = torch.device("cuda:0")
    print("Using GPU")
else:
    device = torch.device("cpu")  # Fallback to CPU
    print("No GPU available, falling back to CPU.")

num_views = 10
hw = 256
H = hw
W = hw
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
    )
    return features.cpu()

In [4]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [ ]:
from diff3f import VERTEX_GPU_LIMIT, batch_render
import random

m = source_mesh

mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
mesh_vertices = mesh.verts_list()[0]

if mesh_vertices is None:
    mesh_vertices = mesh.verts_list()[0]
if len(mesh_vertices) > VERTEX_GPU_LIMIT:
    samples = random.sample(range(len(mesh_vertices)), 10000)
    maximal_distance = torch.cdist(mesh_vertices[samples], mesh_vertices[samples]).max()
else:
    maximal_distance = torch.cdist(mesh_vertices, mesh_vertices).max()  # .cpu()
ball_drop_radius = maximal_distance * tolerance
batched_renderings, normal_batched_renderings, camera, depth = batch_render(
    device, mesh, mesh.verts_list()[0], num_views, H, W, use_normal_map,
    fixed_angle={'type': 'azimuth', 'value': 0}
)
print("Rendering complete")

In [ ]:
import matplotlib.pyplot as plt

def visualize_rendered_views(batched_renderings, num_views):
    # Move to CPU and convert to NumPy
    batched_renderings = batched_renderings.cpu().numpy()

    # Visualize a few views
    for i in range(min(num_views, batched_renderings.shape[0])):
        plt.figure(figsize=(6, 6))
        plt.imshow(batched_renderings[i])
        plt.title(f"Rendered View {i}")
        plt.axis("off")
        plt.show()

# Example Usage
visualize_rendered_views(batched_renderings, num_views)
